# Extracting Blockdata
This script only make sense if you have a running Bitcoin Fullnode. In order to have access to the API, you need to somehow specify three specific parameters:
* The User with which to connect to the Fullnode
* The Password for that user
* The name or IP of the host where the fullnode is running
* the Port (ussually 8332)

By default they will be loaded from environment-variables but you can also hardcode them if you want.

In [1]:
import os
rpc_user = os.getenv('BTC_RPC_USER','bitcoin')
# Or hardcode by uncommenting here:
# rpc_user="bitcoin"
rpc_password = os.getenv('BTC_RPC_PASSWORD')
# Or hardcode by uncommenting here:
# rpc_password="mySecretPassword"
rpc_host = os.getenv('BTC_RPC_HOST','localhost')
# Or hardcode by uncommenting here:
# rpc_host="mybtc-host.local"
rpc_port = os.getenv('BTC_RPC_PORT','8332')
# Or hardcode by uncommenting here:
# rpc_port="1234"

If the connectivity-parameter above are not correct, you'll get an error below. There are more possibilities for issues. Especially if BTC_RPC_HOST is NOT localhost (or 127.0.0.1).
Some resources to read for troubleshooting are:
* https://bitcoin.stackexchange.com/questions/88947/cant-reach-remote-bitcoin-rpc-server
* https://bitcoin.stackexchange.com/questions/91274/bitcoin-json-rpc-not-working-on-remote-ip

Some specific issues:
* _"Name or service not known"_ -> doublecheck rpc_host. Don't add any http(s) or anything like that
* _"ModuleNotFoundError: No module named 'cryptoadvance'"_ --> Check to properly activate the environment
* _"Failed to establish a new connection: [Errno 111] Connection refused'"_ --> are you sure your btc-fullnode is running on rpc_host? If it's a remote-server, maybe there is a firewall in between?
* _"Server responded with error code 401:"_


In [2]:
from cryptoadvance.specter.rpc import BitcoinCLI, RpcError
import numpy as np
cli = BitcoinCLI(rpc_user, rpc_password, rpc_host, rpc_port)
cli.getblockchaininfo()
# To investigate the Bitcoin API, here are some great resources:


{'chain': 'main',
 'blocks': 630128,
 'headers': 630128,
 'bestblockhash': '0000000000000000000ebe91af6cc7f180b0df8a9b48793f723c57f1d218934f',
 'difficulty': 16104807485529.38,
 'mediantime': 1589298250,
 'verificationprogress': 0.9999987399113666,
 'initialblockdownload': False,
 'chainwork': '00000000000000000000000000000000000000000f6956303ef81a92ec6f8bce',
 'size_on_disk': 314522173081,
 'pruned': False,
 'softforks': [{'id': 'bip34', 'version': 2, 'reject': {'status': True}},
  {'id': 'bip66', 'version': 3, 'reject': {'status': True}},
  {'id': 'bip65', 'version': 4, 'reject': {'status': True}}],
 'bip9_softforks': {'csv': {'status': 'active',
   'startTime': 1462060800,
   'timeout': 1493596800,
   'since': 419328},
  'segwit': {'status': 'active',
   'startTime': 1479168000,
   'timeout': 1510704000,
   'since': 481824}},
 'warnings': ''}

The response is encoded in [json](https://en.wikipedia.org/wiki/JSON) (as all the results of bitcoin-rpc-calls) we're looking for is something like:
```
{'chain': 'main',
 'blocks': 629212,
...
```
We can beautify the output a bit like this in order to make it more readable:

In [3]:
import json
print(json.dumps(cli.getblockchaininfo(), indent=2))

{
  "chain": "main",
  "blocks": 630128,
  "headers": 630128,
  "bestblockhash": "0000000000000000000ebe91af6cc7f180b0df8a9b48793f723c57f1d218934f",
  "difficulty": 16104807485529.38,
  "mediantime": 1589298250,
  "verificationprogress": 0.9999987051023334,
  "initialblockdownload": false,
  "chainwork": "00000000000000000000000000000000000000000f6956303ef81a92ec6f8bce",
  "size_on_disk": 314522173081,
  "pruned": false,
  "softforks": [
    {
      "id": "bip34",
      "version": 2,
      "reject": {
        "status": true
      }
    },
    {
      "id": "bip66",
      "version": 3,
      "reject": {
        "status": true
      }
    },
    {
      "id": "bip65",
      "version": 4,
      "reject": {
        "status": true
      }
    }
  ],
  "bip9_softforks": {
    "csv": {
      "status": "active",
      "startTime": 1462060800,
      "timeout": 1493596800,
      "since": 419328
    },
    "segwit": {
      "status": "active",
      "startTime": 1479168000,
      "timeout": 15107

This is the result of the [getblockchaininfo-call](https://bitcoin.org/en/developer-reference#getblockchaininfo). In order to get more knowledge about the Bitcoin-API, especially these three resources might be helpful:
* https://bitcoin.org/en/developer-reference#bitcoin-core-apis -  The reference-documentation. This is the truth by definition
* https://chainquery.com/bitcoin-cli - This is awesome because you can play in a sandbox what all these calls do
* https://github.com/ChristopherA/Learning-Bitcoin-from-the-Command-Line - This is more tutorial style

Let's look into the result of the [getblock-call](https://bitcoin.org/en/developer-reference#getblock). It needs a hash as param and we're using the [getblockhash-call](https://bitcoin.org/en/developer-reference#getblockhash) to obtain the hash via the desired blockheight. Here is an example what gets returned from the 2nd block:

In [4]:
print("The whole json-result looks like this:")
print(json.dumps(cli.getblock(cli.getblockhash(2)), indent=2))

The whole json-result looks like this:
{
  "hash": "000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd",
  "confirmations": 630127,
  "strippedsize": 215,
  "size": 215,
  "weight": 860,
  "height": 2,
  "version": 1,
  "versionHex": "00000001",
  "merkleroot": "9b0fc92260312ce44e74ef369f5c66bbb85848f2eddd5a7a1cde251e54ccfdd5",
  "tx": [
    "9b0fc92260312ce44e74ef369f5c66bbb85848f2eddd5a7a1cde251e54ccfdd5"
  ],
  "time": 1231469744,
  "mediantime": 1231469665,
  "nonce": 1639830024,
  "bits": "1d00ffff",
  "difficulty": 1,
  "chainwork": "0000000000000000000000000000000000000000000000000000000300030003",
  "nTx": 1,
  "previousblockhash": "00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048",
  "nextblockhash": "0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449"
}


So finally, we want to know the flow, the number of generated bitcoins per month. In order to know that we need to know the subsidy of each block and the date. So what we want is a flat-list in order to generate a csv which looks like this:
```
subsidy,date
50,1231469744
... one line per block ...
```
However, the first problem is that there is no subsidy in the data. So we need to calculate that ourself for each block. We know that it's halving every 210000 blocks. With a bit of googling (developers are lazy and you ALWAYS can find the solution to your problem somewhere on the internet) we found:
https://gist.github.com/nelruk/d00d785f84e47288b0ab734d4aab5f49
So this is how our function looks like:

In [5]:
# subsidy is not part of the json. So we need to calclate it ourself
def blocksubsidy(height):
    ' see https://gist.github.com/nelruk/d00d785f84e47288b0ab734d4aab5f49 '
    ' passes in a blockheight and returns the subsidy. E.g. blocksubsidy(5) = 50 '
    period = int(height / 210000)
    subsidy=50
    for i in range(0,period):
        subsidy = subsidy /2
    # easier:
    # subsidy / (2 ^ period)
    return subsidy

print("the blocksubsidy of the 5th      block is: {}".format(blocksubsidy(5)))
print("the blocksubsidy of the 210000th block is: {}".format(blocksubsidy(210000)))
print("the blocksubsidy of the 420000th block is: {}".format(blocksubsidy(420000)))
print("the blocksubsidy of the 630000th block is: {}".format(blocksubsidy(630000)))

the blocksubsidy of the 5th      block is: 50
the blocksubsidy of the 210000th block is: 25.0
the blocksubsidy of the 420000th block is: 12.5
the blocksubsidy of the 630000th block is: 6.25


Back to the csv:
```
subsidy,date
50,1231469744
... one line per block ...
```
The date might looks weird. I took it from the time-field above. It's a [Unix-timestamp](https://en.wikipedia.org/wiki/Unix_time) and we'll later convert it to a human-readable date. But we'll collect more then the minimum. Maybe it will be helpfull to later have ... the difficulty or ... you never know!

So the one thing which is troubling is the "tx":[] field. It's a nested list with tx-IDs and we can't integrate that to our csv because csv-files are "flat", not "nested".
We can explicitely get the keys (which later will get the header to our csv-file) and also the values (which will get the rows.

In [6]:
print("A List of the keys (later the headers):")
print(cli.getblock(cli.getblockhash(2)).keys())
print("----------------------------------------------------")
print("A List of the values (later the rows):")
print(list(cli.getblock(cli.getblockhash(2)).values()))

A List of the keys (later the headers):
dict_keys(['hash', 'confirmations', 'strippedsize', 'size', 'weight', 'height', 'version', 'versionHex', 'merkleroot', 'tx', 'time', 'mediantime', 'nonce', 'bits', 'difficulty', 'chainwork', 'nTx', 'previousblockhash', 'nextblockhash'])
----------------------------------------------------
A List of the values (later the rows):
['000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd', 630127, 215, 215, 860, 2, 1, '00000001', '9b0fc92260312ce44e74ef369f5c66bbb85848f2eddd5a7a1cde251e54ccfdd5', ['9b0fc92260312ce44e74ef369f5c66bbb85848f2eddd5a7a1cde251e54ccfdd5'], 1231469744, 1231469665, 1639830024, '1d00ffff', 1, '0000000000000000000000000000000000000000000000000000000300030003', 1, '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449']


The thing which doesn't work though is the "tx-column" as this value is a list of tx-hashes. We're not interested in that, so let's remove it.

In [7]:
myblock = cli.getblock(cli.getblockhash(1))
del myblock['tx']
np.array(list(myblock.values()))

array(['00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048',
       '630128', '215', '215', '860', '1', '1', '00000001',
       '0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098',
       '1231469665', '1231469665', '2573394689', '1d00ffff', '1',
       '0000000000000000000000000000000000000000000000000000000200020002',
       '1',
       '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f',
       '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd'],
      dtype='<U64')

That looks more like a row in a csv-file. Let's make two functions out of that. this term:
```
blk["subsidy"] = blocksubsidy(height)
```
will call the subsidy-function and simply add the missing piece to the json-data. Then the tx-part will get removed. The block_as_list-function will return a simple array with the values we want in the csv.

In [8]:
def cleansed_block(height):
    ' missing documentation '
    blk = cli.getblock(cli.getblockhash(height))
    blk["subsidy"] = blocksubsidy(height)
    del blk['tx']
    return blk
        
def block_as_list(blk):
        # when loading specify like this:
        # df.columns(['height','subsidy','hash','strippedsize','size','weight','version','versionHex','merkleroot','time','mediantime','nonce','bits','difficulty','chainwork','nTx','previousblockhash','nextblockhash])
        return [blk['height'],blk['subsidy'],blk['hash'],blk['strippedsize'],blk['size'],blk['weight'],blk['version'],blk['versionHex'],blk['merkleroot'],blk['time'],blk['mediantime'],blk['nonce'],blk['bits'],blk['difficulty'],blk['chainwork'],blk['nTx'],blk['previousblockhash'],blk['nextblockhash']]

We're almost done. As writing out all the blocks is taking so much time (10-12h) we're writing it in chunks. Specifying the beginning and end of each file is done in _first_block_ and _last_block_.

In order to track progress, we're printing out a timestamp and the  blockheight every 144 blocks (roughly a day).
The "%" is the rest of a division. So 144%144 == 0!

In [9]:
import csv
import time
from datetime import datetime
number_of_blocks = cli.getblockchaininfo()['blocks']
first_block = 629122
last_block =  number_of_blocks
tic = time.perf_counter()
with open(f"data/blocks_{last_block}.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(first_block, last_block):
        block = cleansed_block(i)
        writer.writerow(block_as_list(block))
        if i % 144 == 0 :
            print(f"{datetime.fromtimestamp(block['time'])} {block['height']}")
toc = time.perf_counter()
print(f"created the csv in {toc - tic:0.4f} seconds")

2020-05-06 04:33:46 629136
2020-05-07 02:08:23 629280
2020-05-08 02:02:18 629424
2020-05-09 03:21:50 629568
2020-05-10 03:32:51 629712
2020-05-11 00:59:59 629856
2020-05-11 21:23:43 630000
created the csv in 123.0787 seconds


It's a bit annoying to manually adjust first_block and last_block and run that script here so often but at least it keeps the code simple. Feel free to improve it!